# Keras Implementation - Human Data Analytics Project 

Importing all modules that we need for our elaboration

In [1]:
import numpy as np
import scipy.io
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score, roc_curve, auc
import matplotlib as plt

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    
    from keras import regularizers
    from keras.layers import Conv1D, BatchNormalization, Dropout, LeakyReLU, Flatten, Activation, Dense, MaxPooling1D
    from keras.models import Model, Sequential
    from keras.optimizers import Adam
    import keras.backend as K
    
K.set_image_data_format('channels_last')

%matplotlib inline

Using TensorFlow backend.


# Preprocessing Section
Importing data sessions for subject 1

In [2]:
x1 = scipy.io.loadmat("data/full/S1-ADL1", mdict={'filled_features':'features', 'labels':'labels'})
x2 = scipy.io.loadmat("data/full/S1-ADL2", mdict={'filled_features':'features', 'labels':'labels'})
x3 = scipy.io.loadmat("data/full/S1-ADL3", mdict={'filled_features':'features', 'labels':'labels'})
x4 = scipy.io.loadmat("data/full/S1-ADL4", mdict={'filled_features':'features', 'labels':'labels'})
x5 = scipy.io.loadmat("data/full/S1-ADL5", mdict={'filled_features':'features', 'labels':'labels'})
x6 = scipy.io.loadmat("data/full/S1-Drill", mdict={'filled_features':'features', 'labels':'labels'})

Check the number of recorded samples for each session

In [3]:
print("Session shapes:\n")
print("ADL1:  ", x1['filled_features'].shape)
print("ADL2:  ", x2['filled_features'].shape)
print("ADL3:  ", x3['filled_features'].shape)
print("ADL4:  ", x4['filled_features'].shape)
print("ADL5:  ", x5['filled_features'].shape)
print("Drill: ", x6['filled_features'].shape)
print("\nTraining set: ADL1 + ADL2 + ADL3 + Drill = ", x1['filled_features'].shape[0] + x2['filled_features'].shape[0] + x3['filled_features'].shape[0] + x6['filled_features'].shape[0])
print("Test set: ADL4 + ADL5 = ", x4['filled_features'].shape[0] + x5['filled_features'].shape[0])

Session shapes:

ADL1:   (51116, 113)
ADL2:   (32224, 113)
ADL3:   (33273, 113)
ADL4:   (32955, 113)
ADL5:   (30127, 113)
Drill:  (54966, 113)

Training set: ADL1 + ADL2 + ADL3 + Drill =  171579
Test set: ADL4 + ADL5 =  63082


We split the data into train and test set.

In [4]:
# features
X_train = np.concatenate((x1['filled_features'],x2['filled_features'],x3['filled_features'],x6['filled_features']),axis=0)
X_test = np.concatenate((x4['filled_features'],x5['filled_features']),axis=0)

# labels (locomotion activity)
Y_train = np.concatenate((x1['labels'][:,0],x2['labels'][:,0],x3['labels'][:,0],x6['labels'][:,0]),axis=0) # here we take just the first column because we want to fulfill the first task
Y_test = np.concatenate((x4['labels'][:,0],x5['labels'][:,0]))
#Y_test = np.concatenate((x4['labels'][:,0],x5['labels'][:,0]),axis=0)

# Initial decision to overcome the problem of entire missing columns
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)


X_train shape:  (171579, 113)
X_test shape:  (63082, 113)


The dataset needs to be normalized in order to improve performances.

In [5]:
# fatures normalization
scaler = StandardScaler().fit(X_train)
X_train =scaler.transform(X_train)
X_test = scaler.transform(X_test)

We decided to use a onehot encoder for the labels

In [6]:
label_max = np.max(Y_train)
print("The higest label value is: ", label_max)
onehot_encoder = OneHotEncoder(n_values=label_max+1, sparse=False)
YOH_train = onehot_encoder.fit_transform(Y_train.reshape(-1, 1))
YOH_test = onehot_encoder.fit_transform(Y_test.reshape(-1, 1))

print("YOH_train shape: ", YOH_train.shape)
print("YOH_test shape: ", YOH_test.shape)

The higest label value is:  5
YOH_train shape:  (171579, 6)
YOH_test shape:  (63082, 6)


In [7]:
window_size = 10
stride = int(window_size / 2)
num_features = X_train.shape[1]
samples, classes = YOH_train.shape
print("Number of samples: ", samples, "\nNumber of classes: ", classes)

windows = int(samples // stride) - 1
print("Number of windows: ", windows)

Number of samples:  171579 
Number of classes:  6
Number of windows:  34314


In [8]:
num_features_test = X_test.shape[1]
samples_test, classes_test = YOH_test.shape
print("Number of samples: ", samples_test, "\nNumber of classes: ", classes_test)

windows_test = int(samples_test // stride) - 1
print("Number of windows: ", windows_test)

Number of samples:  63082 
Number of classes:  6
Number of windows:  12615


In [9]:
cube = np.zeros([windows, window_size, num_features])
cube.shape

(34314, 10, 113)

In [10]:
cube_test = np.zeros([windows_test, window_size, num_features])
cube_test.shape

(12615, 10, 113)

In [11]:
lab_cum = np.zeros([windows])
lab_cum.shape

(34314,)

In [12]:
lab_cum_test = np.zeros([windows_test])
lab_cum_test.shape

(12615,)

In [13]:
for w in range(windows):
    index = int(w * stride)
    cube[w,:,:] = X_train[index:index+window_size, :].reshape((window_size,num_features))
    l = YOH_train[index:index+window_size,:] # shape 50 x 5
    #print(np.sum(l, axis=0),np.argmax(np.sum(l, axis=0)))
    lab_cum[w] = np.argmax(np.sum(l, axis=0))

In [14]:
for w in range(windows_test):
    index = int(w * stride)
    cube_test[w,:,:] = X_test[index:index+window_size, :].reshape((window_size,num_features))
    l = YOH_test[index:index+window_size,:] # shape 50 x 5
    #print(np.sum(l, axis=0),np.argmax(np.sum(l, axis=0)))
    lab_cum_test[w] = np.argmax(np.sum(l, axis=0))

In [15]:
#print('cumulative labels: ', lab_cum.shape, type(lab_cum), "\n", lab_cum)
YOH_train_final = onehot_encoder.fit_transform(lab_cum.reshape(-1, 1))
print(YOH_train_final)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [16]:
YOH_test_final = onehot_encoder.fit_transform(lab_cum_test.reshape(-1, 1))
print(YOH_test_final)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [17]:
def TestModel(input_shape):
    """ 
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns: 
    model -- a Model() instance in Keras
    """
    
    model = Sequential()
    
    model.add(Conv1D(filters = 18,
                    kernel_size=5,
                    strides=1,
                    padding='same',
                    input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling1D(pool_size=2,
                          strides=2,
                          padding='same'))
    
    model.add(Conv1D(filters = 36,
                    kernel_size=7,
                    strides=1,
                    padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling1D(pool_size=2,
                          strides=2,
                          padding='same'))
    
    model.add(Dropout(0.2))
    
    model.add(Conv1D(filters = 72,
                    kernel_size=7,
                    strides=1,
                    padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling1D(pool_size=2,
                          strides=2,
                          padding='same'))
    
    #model.add(Conv1D(filters = 144,
    #                kernel_size=7,
    #                strides=1,
    #                padding='same'))
    #model.add(BatchNormalization())
    #model.add(Activation('relu'))
    #model.add(MaxPooling1D(pool_size=2,
    #                      strides=2,
    #                      padding='same'))
    
    model.add(Flatten())
    
    model.add(Dense(64, kernel_regularizer=regularizers.l2(0.01)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))

    model.add(Dense(classes))
    model.add(Activation('softmax'))
    
    model.summary()
    
    return model

In [18]:
model_test = TestModel((10,113))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 10, 18)            10188     
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 18)            72        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 10, 18)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 5, 18)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 5, 36)             4572      
_________________________________________________________________
batch_normalization_2 (Batch (None, 5, 36)             144       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 5, 36)             0         
__________

In [19]:
opt = Adam(lr=0.01)
model_test.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [20]:
model_test.fit(x = cube, y = YOH_train_final, epochs = 50, batch_size = 128, validation_data=(cube_test,YOH_test_final))

Train on 34314 samples, validate on 12615 samples
Epoch 1/50
34314/34314 [==============================] - 7s 211us/step - loss: 1.2368 - acc: 0.5179 - val_loss: 2.2610 - val_acc: 0.2851
Epoch 2/50
34314/34314 [==============================] - 4s 104us/step - loss: 1.0018 - acc: 0.5848 - val_loss: 3.2455 - val_acc: 0.3725
Epoch 3/50
34314/34314 [==============================] - 3s 101us/step - loss: 0.9578 - acc: 0.6096 - val_loss: 2.4037 - val_acc: 0.2943
Epoch 4/50
34314/34314 [==============================] - 3s 102us/step - loss: 0.9390 - acc: 0.6188 - val_loss: 3.1788 - val_acc: 0.2809
Epoch 5/50
34314/34314 [==============================] - 4s 103us/step - loss: 0.9066 - acc: 0.6331 - val_loss: 2.8871 - val_acc: 0.3738
Epoch 6/50
34314/34314 [==============================] - 3s 102us/step - loss: 0.8897 - acc: 0.6374 - val_loss: 3.1659 - val_acc: 0.3657
Epoch 7/50
34314/34314 [==============================] - 4s 105us/step - loss: 0.8882 - acc: 0.6304 - val_loss: 1.7619 - 

In [39]:
y_pred = model_test.predict(cube_test)
print("y_pred:\n", y_pred)

y_pred_hard = np.argmax(y_pred, axis=1)
print("\ny_pred_hard:\n", y_pred_hard)

YOH_pred = onehot_encoder.fit_transform(y_pred_hard.reshape(-1, 1))
print("\nYOH_pred:\n", YOH_pred)

y_pred:
 [[2.74701668e-11 9.49683905e-01 5.03114127e-02 1.09776814e-11
  4.77086496e-06 9.86764892e-10]
 [1.04694731e-09 8.97237480e-01 1.02724105e-01 6.62097210e-10
  3.83340375e-05 8.92026435e-08]
 [1.04239412e-07 6.94116533e-01 3.05433244e-01 1.23025686e-07
  4.25656355e-04 2.44067105e-05]
 ...
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]

y_pred_hard:
 [1 1 1 ... 2 2 2]

YOH_pred:
 [[0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [40]:
print("F1-measure: ", f1_score(YOH_test_final, YOH_pred, average='weighted'))

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(classes):
    fpr[i], tpr[i], _ = roc_curve(YOH_test_final[:, i], YOH_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

print("ROC-AUC measure: ", roc_auc)

F1-measure:  0.9199016588917385
ROC-AUC measure:  {0: nan, 1: 0.9240272828708203, 2: 0.9053215212327809, 3: nan, 4: 0.9960707019522338, 5: 0.9264151251182668}


C:\Users\Riccardo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Riccardo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Riccardo\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
